From https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/label_image/label_image.py

you can run the script to get result

```
python label_image.py --image=image_path \
    --graph=graph_path \
    --labels=labels.txt \ 
    --output_layer=MobilenetV1/Predictions/Reshape_1
```

Before run the script, you have to run Freeze Graph Script.

In [1]:
import tensorflow as tf
import numpy as np

slim = tf.contrib.slim

In [2]:
image_file = "/notebooks/images/car.jpg"
graph_file = "./.mobilenet_v1/mobilenet_v1.pb"
label_file = "./.mobilenet_v1/labels.txt"
input_node_name = "import/input"
output_node_name = "import/MobilenetV1/Predictions/Reshape_1"

In [3]:
def load_graph():
    graph = tf.Graph()
    graph_def = tf.GraphDef()

    with open(graph_file, "rb") as f:
        graph_def.ParseFromString(f.read())
    with graph.as_default():
        tf.import_graph_def(graph_def)

    return graph

In [4]:
def read_tensor_from_image_file(input_height=224, input_width=224,
                                input_mean=0, input_std=255):
    input_name = "file_reader"
    output_name = "normalized"
    file_reader = tf.read_file(image_file, input_node_name)
    if image_file.endswith(".png"):
        image_reader = tf.image.decode_png(file_reader, channels=3,
                                           name='png_reader')
    elif image_file.endswith(".gif"):
        image_reader = tf.squeeze(tf.image.decode_gif(file_reader,
                                                      name='gif_reader'))
    elif image_file.endswith(".bmp"):
        image_reader = tf.image.decode_bmp(file_reader, name='bmp_reader')
    else:
        image_reader = tf.image.decode_jpeg(file_reader, channels=3,
                                            name='jpeg_reader')
        
    float_caster = tf.cast(image_reader, tf.float32)
    dims_expander = tf.expand_dims(float_caster, 0)
    resized = tf.image.resize_bilinear(
        dims_expander, [input_height, input_width])
    normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
    sess = tf.Session()
    result = sess.run(normalized)

    return result

In [5]:
def load_labels():
    label = []
    proto_as_ascii_lines = tf.gfile.GFile(label_file).readlines()
    for l in proto_as_ascii_lines:
        label.append(l.rstrip())
    return label

In [7]:
graph = load_graph()
input_operation = graph.get_operation_by_name(input_node_name)
output_operation = graph.get_operation_by_name(output_node_name)

image = read_tensor_from_image_file()


with tf.Session(graph=graph) as sess:
    probabilities = sess.run(output_operation.outputs[0],
                           {input_operation.outputs[0]: image})
    
    results = np.squeeze(probabilities)
    
top_k = results.argsort()[-5:][::-1]

labels = load_labels()
for i in top_k:
    print(labels[i], results[i])

818:sports car, sport car 0.932937
752:racer, race car, racing car 0.0281193
512:convertible 0.0192136
437:beach wagon, station wagon, wagon, estate car, beach waggon, station waggon, waggon 0.0091449
582:grille, radiator grille 0.0074993
